# Cartographic Rendering

Display a geospatial dataset with lat/lon coordinates in a cartographic 3D reference with [GeoVista](https://github.com/bjlittle/geovista)

Requires the following pull requests:
    
- PyVista algorithm support: https://github.com/pyvista/pyvista/pull/3318
- GeoVista algorithm support: https://github.com/bjlittle/geovista/pull/127
- PyVista NaN opacity: https://github.com/pyvista/pyvista/pull/3556
- Better support with trame: https://github.com/pyvista/pyvista/pull/3385

In [ ]:
import geovista as gv
import pyvista as pv
from pvxarray.vtk_source import PyVistaXarraySource
import xarray as xr

In [ ]:
# Requires https://github.com/pyvista/pyvista/pull/3385
pv.set_jupyter_backend("trame-server")

In [ ]:
# Download https://www.ncei.noaa.gov/products/optimum-interpolation-sst
ds = xr.open_dataset("oisst-avhrr-v02r01.19810901.nc")
da = ds.err
source = PyVistaXarraySource(da, x="lon", y="lat", z="zlev", time="time", resolution=1.0)

In [ ]:
plotter = gv.GeoPlotter()
# Requires https://github.com/pyvista/pyvista/pull/3318 and https://github.com/bjlittle/geovista/pull/127
actor = plotter.add_mesh(
    source,
    cmap="coolwarm",
    show_edges=True,
    # Requires https://github.com/pyvista/pyvista/pull/3556
    nan_opacity=0,
)
basemap_actor = plotter.add_base_layer(texture=gv.blue_marble())
resolution = "10m"
plotter.add_coastlines(resolution=resolution, color="white")
plotter.enable_depth_peeling()
plotter.view_isometric()
plotter.show()